In [11]:
import argparse
from types import SimpleNamespace
import sys
sys.path.append('/data/lige/HKN')# Please change accordingly!

from __future__ import division
from __future__ import print_function

from geoopt import ManifoldParameter as geoopt_ManifoldParameter
from manifolds.base import ManifoldParameter as base_ManifoldParameter

import datetime
import json
import logging
from optim import RiemannianAdam, RiemannianSGD
import os
import pickle
import time

import numpy as np
import torch
from config import parser
from models.base_models import NCModel, LPModel, GCModel
from utils.data_utils import load_data, get_nei, GCDataset, split_batch
from utils.train_utils import get_dir_name, format_metrics
from utils.eval_utils import acc_f1

from geoopt import ManifoldParameter as geoopt_ManifoldParameter
from manifolds.base import ManifoldParameter as base_ManifoldParameter

import torch.nn.functional as F


config_args = {
    'training_config': {
        'lr': (1e-3, 'learning rate'),
        'dropout': (0.25, 'dropout probability'),
        'cuda': (0, 'which cuda device to use (-1 for cpu training)'),
        'epochs': (1000, 'maximum number of epochs to train for'),
        'weight_decay': (1e-3, 'l2 regularization strength'),
        'optimizer': ('radam', 'which optimizer to use, can be any of [rsgd, radam]'),
        'momentum': (0.999, 'momentum in optimizer'),
        'patience': (15, 'patience for early stopping'),
        'seed': (1234, 'seed for training'),
        'log_freq': (5, 'how often to compute print train/val metrics (in epochs)'),
        'eval_freq': (1, 'how often to compute val metrics (in epochs)'),
        'save': (0, '1 to save model and logs and 0 otherwise'),
        'save_dir': (None, 'path to save training logs and model weights (defaults to logs/task/date/run/)'),
        'sweep_c': (0, ''),
        'lr_reduce_freq': (None, 'reduce lr every lr-reduce-freq or None to keep lr constant'),
        'gamma': (0.5, 'gamma for lr scheduler'),
        'print_epoch': (True, ''),
        'grad_clip': (None, 'max norm for gradient clipping, or None for no gradient clipping'),
        'min_epochs': (300, 'do not early stop before min-epochs')
    },
    'model_config': {
        'use_geoopt': (False, "which manifold class to use, if false then use basd.manifold"),
        'AggKlein':(False, "if false, then use hyperboloid centorid for aggregation"),
        'corr': (0,'0: d(x_i ominus x, x_k), 1: d(x_ik,x_k)'),
        'task': ('nc', 'which tasks to train on, can be any of [lp, nc]'),
        'model': ('BKNet', 'which encoder to use, can be any of [Shallow, MLP, HNN, GCN, GAT, HyperGCN, HyboNet,BKNet,BMLP]'),
        'dim': (64, 'embedding dimension'),
        'manifold': ('PoincareBall', 'which manifold to use, can be any of [Euclidean, Hyperboloid, PoincareBall, Lorentz]'),
        'c': (1.0, 'hyperbolic radius, set to None for trainable curvature'),
        'r': (2., 'fermi-dirac decoder parameter for lp'),
        't': (1., 'fermi-dirac decoder parameter for lp'),
        'margin': (2., 'margin of MarginLoss'),
        'pretrained_embeddings': (None, 'path to pretrained embeddings (.npy file) for Shallow node classification'),
        'pos_weight': (0, 'whether to upweight positive class in node classification tasks'),
        'num_layers': (2, 'number of hidden layers in encoder'),
        'bias': (1, 'whether to use bias (1) or not (0)'),
        'act': ('relu', 'which activation function to use (or None for no activation)'),
        'n_heads': (4, 'number of attention heads for graph attention networks, must be a divisor dim'),
        'alpha': (0.2, 'alpha for leakyrelu in graph attention networks'),
        'double_precision': ('1', 'whether to use double precision'),
        'use_att': (0, 'whether to use hyperbolic attention or not'),
        'local_agg': (0, 'whether to local tangent space aggregation or not'),
        'kernel_size': (4, 'number of kernels'),
        'KP_extent': (0.66, 'influence radius of each kernel point'),
        'radius': (1, 'radius used for kernel point init'),
        'deformable': (False, 'deformable kernel'),
        'linear_before': (64, 'dim of linear before gcn')#64
    },
    'data_config': {
        'dataset': ('wisconsin', 'which dataset to use(cornell,wisconsin,squirrel,cora)'),
        'batch_size': (32, 'batch size for gc'),
        'val_prop': (0.05, 'proportion of validation edges for link prediction'),
        'test_prop': (0.1, 'proportion of test edges for link prediction'),
        'use_feats': (1, 'whether to use node features or not'),
        'normalize_feats': (1, 'whether to normalize input node features'),
        'normalize_adj': (1, 'whether to row-normalize the adjacency matrix'),
        'split_seed': (1234, 'seed for data splits (train/test/val)'),
        'split_graph': (False, 'whether to split the graph')
    }
}

# 将所有参数转换为 SimpleNamespace
args = SimpleNamespace(
    **{k: v[0] for config in config_args.values() for k, v in config.items()}
)

#choose which manifold class to follow 
if args.use_geoopt == False:
    ManifoldParameter = base_ManifoldParameter
else:
    ManifoldParameter = geoopt_ManifoldParameter
np.random.seed(args.seed)#args.seed
torch.manual_seed(args.seed)#args.seed
if int(args.cuda):#args.double_precision
    torch.set_default_dtype(torch.float64)
if int(args.cuda) >= 0:#args.cuda
    torch.cuda.manual_seed(args.seed)#args.seed
args.device = 'cuda:' + str(args.cuda) if int(args.cuda) >= 0 else 'cpu' #args.device actually,<-args.cuda
args.patience = args.epochs if not args.patience else args.patience #args.patience<-args.epochs|args.patience

print(f'Using: {args.device}')
print("Using seed {}.".format(args.seed))
print(f"Dataset: {args.dataset}")

# Load data
data = load_data(args, os.path.join('data', args.dataset))
if args.task == 'gc':
    args.n_nodes, args.feat_dim = data['features'][0].shape
else:
    args.n_nodes, args.feat_dim = data['features'].shape
if args.task == 'nc':
    Model = NCModel
    args.n_classes = int(data['labels'].max() + 1)
    args.data = data
    print(f'Num classes: {args.n_classes}')
elif args.task == 'gc':
    Model = GCModel
    args.n_classes = int(data['labels'].max() + 1)
    print(f'Num classes: {args.n_classes}')
else:
    args.nb_false_edges = len(data['train_edges_false'])
    args.nb_edges = len(data['train_edges'])
    if args.task == 'lp':
        Model = LPModel
        args.n_classes = 2

if not args.lr_reduce_freq:
    args.lr_reduce_freq = args.epochs


###A simple check on data
print(data.keys())
print(data['adj_train'].todense().shape)
print(data['features'].shape)
###A simple check on data

# Model and optimizer
model = Model(args)
print(str(model))
no_decay = ['bias', 'scale']
optimizer_grouped_parameters = [{
    'params': [
        p for n, p in model.named_parameters()
        if p.requires_grad and not any(
            nd in n
            for nd in no_decay) and not isinstance(p, ManifoldParameter)
    ],
    'weight_decay':
    args.weight_decay
}, {
    'params': [
        p for n, p in model.named_parameters() if p.requires_grad and any(
            nd in n
            for nd in no_decay) or isinstance(p, ManifoldParameter)
    ],
    'weight_decay':
    0.0
}]
if args.optimizer == 'radam':
    optimizer = RiemannianAdam(params=optimizer_grouped_parameters,
                                lr=args.lr,
                                stabilize=10)
elif args.optimizer == 'rsgd':
    optimizer = RiemannianSGD(params=optimizer_grouped_parameters,
                                lr=args.lr,
                                stabilize=10)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=int(
                                                args.lr_reduce_freq),
                                                gamma=float(args.gamma))
tot_params = sum([np.prod(p.size()) for p in model.parameters()])
model = model.to(args.device)
for x, val in data.items():
    if torch.is_tensor(data[x]):
        data[x] = data[x].to(args.device)
print(f"Total number of parameters: {tot_params}")

# Train model for nc:
t_total = time.time()
counter = 0
best_val_metrics = model.init_metric_dict()
best_test_metrics = None
best_emb = None
if args.n_classes > 2:
    f1_average = 'micro'
else:
    f1_average = 'binary'

if args.model == 'HKPNet':
    nei, nei_mask = get_nei(data['adj_train'])
    nei = nei.to(args.device)
    nei_mask = nei_mask.to(args.device)
elif args.model == 'BKNet':
    nei, nei_mask = get_nei(data['adj_train'])
    nei = nei.to(args.device)
    nei_mask = nei_mask.to(args.device) #nei/nei_mask on cuda now



Using: cuda:0
Using seed 1234.
Dataset: wisconsin
Num classes: 5
dict_keys(['adj_train', 'features', 'labels', 'idx_train', 'idx_val', 'idx_test', 'adj_train_norm'])
(251, 251)
torch.Size([251, 1703])
AggKlein == False
corr == 0
NCModel(
  (encoder): BKNet(
    (linear_before): BLinear(in_features=1703, out_features=64, c=tensor([1.], device='cuda:0'), use_bias=True, act=None)
    (layers): Sequential(
      (0): KPGraphConvolution(
        (net): KernelPointAggregation(
          (linears): ModuleList(
            (0): BLinear(in_features=64, out_features=64, c=tensor([1.], device='cuda:0'), use_bias=1, act=None)
            (1): BLinear(in_features=64, out_features=64, c=tensor([1.], device='cuda:0'), use_bias=1, act=None)
            (2): BLinear(in_features=64, out_features=64, c=tensor([1.], device='cuda:0'), use_bias=1, act=None)
            (3): BLinear(in_features=64, out_features=64, c=tensor([1.], device='cuda:0'), use_bias=1, act=None)
          )
          (MLP_f): BMLP(
  

/data/lige/HKN/utils/data_utils.py:347: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(G, sorted(G.nodes()))


In [12]:
model.encoder

BKNet(
  (linear_before): BLinear(in_features=1703, out_features=64, c=tensor([1.], device='cuda:0'), use_bias=True, act=None)
  (layers): Sequential(
    (0): KPGraphConvolution(
      (net): KernelPointAggregation(
        (linears): ModuleList(
          (0): BLinear(in_features=64, out_features=64, c=tensor([1.], device='cuda:0'), use_bias=1, act=None)
          (1): BLinear(in_features=64, out_features=64, c=tensor([1.], device='cuda:0'), use_bias=1, act=None)
          (2): BLinear(in_features=64, out_features=64, c=tensor([1.], device='cuda:0'), use_bias=1, act=None)
          (3): BLinear(in_features=64, out_features=64, c=tensor([1.], device='cuda:0'), use_bias=1, act=None)
        )
        (MLP_f): BMLP(
          (linear1): BLinear(in_features=64, out_features=128, c=tensor([1.], device='cuda:0'), use_bias=1, act=None)
          (act): BAct(c=tensor([1.], device='cuda:0'), act=<function relu at 0x7f8560393370>)
          (linear2): BLinear(in_features=128, out_features=128,

In [13]:
import layers.B_layers as Blayers
x_nei = Blayers.gather(data['features'], nei) # (n, nei_num, d)
print(x_nei.shape)

torch.Size([251, 122, 1703])


In [14]:
klein_points = model.manifold.klein_proj(model.manifold.poincare_to_klein(x_nei,model.c), model.c)
target_klein_point = model.manifold.klein_proj(model.manifold.klein_midpoint(klein_points), model.c)
target_poincare_point = model.manifold.proj(model.manifold.klein_to_poincare(target_klein_point, model.c), model.c)
target_poincare_point.shape

torch.Size([251, 1703])

In [15]:
norm=torch.norm(target_poincare_point, dim=-1)#
# 将 norm 调整为 [251, 1]
norm = norm.unsqueeze(1)  # 形状: [251, 1]

# 确保 norm 中没有零值，避免除零错误
norm = torch.clamp(norm, min=1e-6)
print(norm.shape)
cond = norm > 0.5
projected = target_poincare_point / norm * 0.5

torch.Size([251, 1])


In [18]:
res=model.manifold.proper_tangent_space(x_nei, model.c)
print(res.shape,x_nei.shape)
torch.norm(res, dim=-1)


torch.Size([251, 1703]) torch.Size([251, 122, 1703])


tensor([0.1148, 0.1179, 0.1170, 0.1164, 0.1171, 0.1172, 0.1171, 0.1179, 0.1179,
        0.1179, 0.1179, 0.1164, 0.1171, 0.1179, 0.1179, 0.1179, 0.1179, 0.1179,
        0.1179, 0.1179, 0.1151, 0.1174, 0.1167, 0.1179, 0.1176, 0.1165, 0.1171,
        0.1164, 0.1164, 0.1164, 0.1164, 0.1179, 0.1157, 0.1179, 0.1163, 0.1173,
        0.1179, 0.1155, 0.1172, 0.1170, 0.1179, 0.1164, 0.1179, 0.1170, 0.1117,
        0.1157, 0.1156, 0.1172, 0.1167, 0.1164, 0.1140, 0.1167, 0.1179, 0.1160,
        0.1127, 0.1179, 0.1179, 0.1164, 0.1171, 0.1179, 0.1179, 0.1179, 0.1179,
        0.1164, 0.1159, 0.1179, 0.1174, 0.1179, 0.1171, 0.1158, 0.1164, 0.1164,
        0.1179, 0.1179, 0.1173, 0.1128, 0.1179, 0.1179, 0.1172, 0.1124, 0.1179,
        0.1157, 0.1171, 0.1169, 0.1179, 0.1179, 0.1164, 0.1163, 0.1179, 0.1155,
        0.1174, 0.1160, 0.1148, 0.1179, 0.1179, 0.1179, 0.1163, 0.1162, 0.0663,
        0.1179, 0.1179, 0.1157, 0.1165, 0.1158, 0.1179, 0.1179, 0.1171, 0.1151,
        0.1171, 0.1179, 0.1171, 0.1179, 

In [17]:
model.manifold.proper_tangent_space(data['features'], model.c)

tensor([1.0113e-04, 3.9234e-05, 1.8320e-05,  ..., 1.4654e-04, 1.1818e-02,
        1.3010e-04], device='cuda:0')